In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.interpolate import barycentric_interpolate
from numpy import linalg as la
from scipy.optimize import root

# Problem 1

In [ ]:
c = lambda x:-.7*(x**2-1)
alpha = lambda x:1/np.sqrt(1-c(x)**2)

In [ ]:
def L_(y, dy, x):
    L = alpha(x)*np.sqrt(1+(alpha(x)*dy(x))**2)-alpha(x)**2*c(x)*dy(x)
    return L

In [ ]:
def T(y,dy,x0,x1,N):
    x_domain = np.linspace(x0,x1,N+1)
    h = 1.*(x1-x0)/N
    T = h*np.sum([L_(y,dy,x_domain[i]) for i in xrange(N+1)])
    return T

# Problem 2

In [ ]:
y = lambda x:2.5*(x+1)
dy = lambda x:2.5

In [ ]:
print 'upper bound: ' + str(T(y,dy,-1,1,100000))

In [ ]:
print 'lower bound: 2'

# Problem 3

In [ ]:
def cheb(N):
    x = np.cos((np.pi/N)*np.linspace(0,N,N+1))
    x.shape = (N+1,1)
    lin = np.linspace(0,N,N+1)
    lin.shape = (N+1,1)
    
    c = np.ones((N+1,1))
    c[0], c[-1] = 2., 2.
    c = c*(-1.)**lin
    X = x*np.ones(N+1) # broadcast along 2nd dimension (columns)
    
    dX = X - X.T
    
    D = (c*(1./c).T)/(dX + np.eye(N+1))
    D = D - np.diag(np.sum(D.T,axis=0))
    x.shape = (N+1,)
    # Here we return the differentiation matrix and the Chebyshev points,
    # numbered from x_0 = 1 to x_N = -1
    return D, x

In [ ]:
N = 25
D, x = cheb(N)

In [ ]:
def Newton(Y):
    dY = D.dot(Y)
    a = alpha(x)
    out = D.dot(a**3*dY/np.sqrt((1+(a*dY)**2)) - a**2*c(x))
    out[0] = Y[0]-5
    out[-1] = Y[-1]
    return out

In [ ]:
guess = 2.5*(x+1)
sol = root(Newton, guess).x

In [ ]:
lin = np.linspace(-1,1,100)
barycentric = barycentric_interpolate(x, sol, lin)
plt.plot(lin, barycentric)
plt.plot(lin, 2.5*(lin+1))
plt.ylim((-1,5))
plt.show()



# Problem 4

In [ ]:
theta = []
dy = D.dot(sol)

def find_theta(theta_i):
    return dy[i]-c(x[i])/np.cos(theta_i)-np.tan(theta_i)

for i in xrange(N+1):
    theta_i = root(find_theta, np.pi/4).x
    theta.append(theta_i)

In [ ]:
plt.plot(x,theta)
plt.xlim((-1,1))
plt.ylim((0,np.pi/2))
plt.show()

In [ ]:
lin = np.linspace(-1,1,100)
barycentric = barycentric_interpolate(x, theta, lin)

plt.plot(lin,barycentric)
plt.xlim((-1,1))
plt.ylim((0,np.pi/2))
plt.xlabel('x')
plt.ylabel(r'$\theta$')
plt.yticks((0, np.pi/6., np.pi/3., np.pi/2.),
        ('0', r'$\frac{\pi}{6}$', r'$\frac{\pi}{3}$', r'$\frac{\pi}{2}$'))
plt.show()